## Logging

DEBUG/INFO/WARN/FATAL

Il faut définir un fichier de configuration : configuration des loggers, définition des appenders, association des appenders aux loggers avec un layout. Dans le code source des classes, il faut :
* obtenir une instance du logger relative à la classe
* utiliser l'API pour émettre un message associé à un niveau de gravit

Ex : 
log4j.rootLogger=DEBUG, stdout
log4j.appender.stdout=org.apache.log4j.ConsoleAppender
log4j.appender.stdout.layout=org.apache.log4j.PatternLayout
log4j.appender.stdout.layout.ConversionPattern=%d [%-5p] (%F:%M:%L) %m%n

Cette configuration définit le niveau de gravité DEBUG pour le logger racine et lui associe un logger nommé arbitrairement stdout. Par héritage, tous les loggers de l'application vont hériter de cette configuration.

L'appender nommé stdout est de type ConsoleAppender : il envoie les messages sur la console standard.

Un layout personnalisé est associé à l'appender nommé stdout pour formater les messages.

package com.jmdoudoux.test.log4j;

import org.apache.log4j.Logger;

public class TestLog4j1 {

  // Il est possible de donner un nom arbitraire au Logger, cependant, comme nous le verrons lorsque nous parlerons des //hiérarchies de Loggers et de la configuration, il est préférable d'utiliser le nom de la classe pour des raisons de facilité.
//private static Logger logger = Logger.getLogger(TestLog4j1.class);
  
  public static void main(String[] args) {
    logger.debug("msg de debogage");
    logger.info("msg d'information");
    logger.warn("msg d'avertissement");
    logger.error("msg d'erreur");
    logger.fatal("msg d'erreur fatale");   
  }
}

Le principe central de la configuration est sa hiérarchisation, c'est-à-dire que chaque Logger va hériter du niveau de son plus proche parent pour lequel on en a défini un, à moins qu'on ne lui attribue explicitement un niveau. Un Logger est parent d'un autre si son nom complet est le préfixe du second. Par exemple, le Loggercom.developpez.beuss.java sera le parent du Loggercom.developpez.beuss.java.log4j, il lui transmettra donc son niveau. Il existe un Logger particulier appelé rootLogger (Logger racine) qui est l'ancêtre de tous les Loggers existants.

log4j.logger.nom.du.logger=[niveau], appender1, appender2

L'intérêt de ce mécanisme est évident lorsque l'on utilise différentes bibliothèques au sein d'un projet qui se servent de Log4j pour journaliser les messages. Il est possible de compartimenter la configuration pour, par exemple, avoir tous les messages de débogage des classes que l'on développe mais uniquement les messages d'erreur des bibliothèques utilisées.

Logger	Niveau
root	error (assigné)
org	error (hérité)
org.apache	error (hérité)
org.apache.commons	error (hérité)
org.apache.catalina	info (assigné)
org.apache.catalina.core	info (hérité)
com.developpez.beuss.java	debug (assigné)
com.developpez.beuss.java.log4j	debug (hérité)

Additivité des Appenders▲
De prime abord, le fonctionnement de la configuration des Appenders peut sembler similaire à celui des niveaux, mais ce n'est pas le cas. En effet, il ne s'agit plus d'héritage proprement dit mais d'additivité. C'est-à-dire qu'un Logger d'un niveau donné va bénéficier de tous les Appenders de ses ancêtres en plus de ceux qui lui sont éventuellement affectés. Il est possible de "couper" la chaîne à un niveau quelconque en indiquant que l'on ne veut pas que l'additivité s'applique. Un exemple vaut mieux qu'un long discours :

Logger	Appender(s)	Additivité
root	RootAppender	Non Pertinent
org	RootAppender (hérité), OrgAppender (assigné)	Oui
org.apache	RootAppender (hérité), OrgAppender (hérité)	Oui
org.apache.commons	RootAppender (hérité), OrgAppender (hérité)	Oui
org.apache.catalina	CatalinaAppender (assigné)	Non
org.apache.catalina.core	CatalinaAppender (hérité)	Oui
com.developpez.beuss.java	RootAppender (hérité), BeussJavaAppender (assigné)	Oui
com.developpez.beuss.java.log4j	RootAppender (hérité), BeussJavaAppender (assigné)	Oui
Avec ce type de configuration, on peut décider par exemple que le Logger racine enverra ses messages vers la console et que certains Loggers journaliseront dans un fichier. Le Logger org.apache.catalina introduit une rupture dans l'additivité, seuls les Appenders définis à partir de son niveau seront affectés à ses descendants.

Il faut savoir qu'au niveau du fichier de configuration, chaque Appender doit avoir un nom afin de pouvoir y faire référence lors de la configuration des Loggers. Les paramètres de configuration concernant les Appenders sont préfixés par log4j.appender. La déclaration d'un Appender d'un nom donné se fait de la façon suivante :

log4j.appender.NomAppender=ClasseAppender
log4j.appender.NomAppender.NomOption=ValeurOption

log4j.appender.Console=org.apache.log4j.ConsoleAppender
log4j.appender.Console.layout=org.apache.log4j.SimpleLayout
Une fois les Appenders configurés, il vous reste à configurer les Loggers pour qu'ils les utilisent.

La forme de la configuration des Loggers est similaire à celle des Appenders, néanmoins, les seules choses à configurer sont les Appenders et, éventuellement, le niveau et l'additivité des Appenders. Aussi la configuration d'un Logger a l'aspect suivant :
log4j.logger.nom.du.logger=[niveau], appender1, appender2

nom.du.logger est le nom du Logger lorsqu'il est demandé viaLogger.getLogger (vous pouvez ne spécifier qu'une partie du nom du Logger et vous reposer sur les mécanismes d'héritage de propriétés) 

La seule particularité concerne la configuration du Logger racine. En effet, celui-ci n'a pas de nom attribué puisqu'il n'est accessible que par la méthode Logger.getRootLogger(). Aussi, il existe une clé particulière servant à sa configuration qui est log4j.rootLogger

Le paramétrage de l'additivité d'un Logger en ce qui concerne les Appenders se fait de façon quelque peu contre-intuitive. Alors que l'on pourrait penser que l'additivité est un paramètre du Logger et qu'à ce titre elle pourrait être configurée selon le même schéma que les paramètres des Appenders, l'équipe de log4j n'a pas fait ce choix. La configuration de l'additivité se fait de la façon suivante :

log4j.additivity.nom.du.logger=true|false
Par défaut, l'additivité est active.

Le ConsoleAppender journalise, comme son nom l'indique, vers la console. Il est cependant possible de choisir si les messages sont envoyés vers la sortie standard ou vers la sortie des erreurs. Ce comportement est contrôlé par le paramètre target qui prend les valeurs System.out (sortie standard) ou System.err (sortie des erreurs).

Comme nous l'avons vu lors de l', il est possible de définir un seuil au niveau de l'Appender en dessous duquel les messages ne seront pas journalisés. Il vient en complément du seuil global et du niveau propre du chaque Logger. Ce seuil est positionnable via la propriété threshold

Néanmoins, il peut être souhaitable de réellement séparer les différents niveaux pour, par exemple, avoir un fichier contenant les messages de débogage, un autre les messages d'information, etc. Dans ce cas, le paramètre threshold n'est plus suffisant. Pour cela, il faut utiliser un autre mécanisme de log4j : les filtres.

https://medium.com/@iacomini.riccardo/spark-logging-configuration-in-yarn-faf5ba5fdb01
https://veerasundar.com/blog/2009/08/log4j-tutorial-additivity-what-and-why/
https://medium.com/@shantanualshi/logging-in-pyspark-36b0bd4dec55

## Spark submit

```bash
spark-submit \
  --name myApp \
  --master yarn \
  --deploy-mode cluster \
  --queue myQueue \
  --conf spark.driver.extraJavaOptions="-Dlog4j.configuration=file:/path/to/log4j-spark.properties" \
  --conf spark.executor.extraJavaOptions="-Dlog4j.configuration=file:/path/to/log4j-spark.properties" \
  mySparkApp.py
```
Est équivalent à
```bash
spark-submit \
  --name myApp \
  --master yarn \
  --deploy-mode cluster \
  --queue myQueue \
  --files /path/to/log4j-spark.properties \
  --conf spark.driver.extraJavaOptions="-Dlog4j.configuration=log4j-spark.properties" \
  --conf spark.executor.extraJavaOptions="-Dlog4j.configuration=log4j-spark.properties" \
  mySparkApp.py
```

Le second est plus propre car moins ambigü. Dans la première solution, on ajoute ```file:``` pour signaler que le fichier se trouve sur le file system local d'où on lance l'application (sinon chaque exécuteur / driver va chercher le fichier sans un sous-répertoire de son working directory qui n'existe pas). La seconde solution fait apparaitre clairement que le fichier log4j-spark.properties est placé dans le working directory des exécuteurs / driver, d'où le chemin allégé passé en option spark.driver.extraJavaOptions et spark.executor.extraJavaOptions.

## Hive support en cluster mode

Il semble que sur notre cluster, la config Hive soit manquante sur les datanodes. On doit donc la passer manuellement à Spark : 

```bash
spark-submit \
    --name myApp \
	--master yarn \
	--deploy-mode cluster \
	--queue myQueue \
	--files /path/to/log4j.properties,/etc/spark2/conf/hive-site.xml \
	--conf spark.driver.extraJavaOptions='-Dlog4j.configuration=log4j_testApp.properties' \
	--conf spark.executor.extraJavaOptions='-Dlog4j.configuration=log4j_testApp.properties' \
	mySparkApp.py
```

## Using Scala UDF in Pyspark

https://fr.slideshare.net/SparkSummit/understanding-memory-management-in-spark-for-fun-and-profit
https://medium.com/wbaa/using-scala-udfs-in-pyspark-b70033dd69b9

if you are using Python, since Python will be all off-heap memory and would not use the ram we reserved for heap. So, by decreasing this value, you reserve less space for the heap, thus you get more space for the off-heap operations (we want that, since Python will operate there). The java process is what uses heap memory, the python process uses off heap. (peut être important notamment pour le driver qui exécute à coup sûr du python pour une app Pyspark).

## Logging Python

### Module ```logging```

Intégré à la librairie standard de Python, il fournit classes et fonctions nécessaires à l'implémentation d'un système de logging pour les applications Python. 

Remarques : 
* ```print``` redirige vers la sortie standard / vers la console.
* Il existe également une librairie ```warnings``` qui permet d'émettre des *warnings* via ```warnings.warn()```. Un tel *warning* est à différencier du *warning* de ````logging``` (```Logger.warning()``` / ```logging.warning()```). Dans le premier cas, le *warning* sert à attirer l'attention du client sur l'existance d'une situation évitable dans son code et que ce dernier doit et peut modifier. Par exemple, un *depreciation warning* va inviter le client à utiliser une autre fonction. Dans le second cas, le client n'a pas forcément la possibilité de modifier son code mais doit être tout de même averti de la situation qui s'est présentée à l'exécution. Dans ce dernier ca, le *warning* n'est qu'un log avec un degré de sévérité particulier.

### Les 4 éléments principaux du système de *logging*
Le système de *logging* se repose sur 2 types d'objets principaux qu'on retrouve aussi dans les systèmes de *logging* d'autres langages (ex: Log4j) : 
* Les Loggers (```logging.Logger```) : ils correspondent à l'interface que l'application utilise directement pour *logger*. Les Loggers sont les objets utilisé pour émettre un message via un appel à l'une de leurs méthodes(```debug```, ```info```, etc.).
* Les Handlers (```logging.Handler```) : attachés à des *loggers*, ils formattent et émettent les messages créés par les *loggers* vers la destination appropriée : sortie d'erreur ou standard, fichier, serveur, etc. Ce sont les *handlers* qui définissent la destination finale des logs.
A ces deux objets centraux on peut ajouter :
* Les Formatters (```logging.Formatter```) : attachés à des *handlers*, ils permettent de spécifier l'apparence des messages finalement émis par leur *handler*.
* Les Filters (```logging.Filter```) : attachés à des *loggers* ou des *handlers*, ils donnent un contrôle plus fin sur les messages qu'on souhaite finalement émis. 

Loggers et handlers ont de commun de chacun posséder un niveau de sévérité et d'éventuels filtres. Niveau de sévérité et filtres ont pour vocation de permettre un contrôle fin des messages qu'on souhaite finalement émis. Le module ```logging``` offre 5 niveaux de sévérité : ```DEBUG```, ```INFO```, ```WARNING```, ```ERROR``` et ```CRITICAL``` par ordre de gravité (on remarque qu'on a pas l'équivalement du niveau TRACE comme en Java par exemple). Chacun de ces niveau est associé à un nombre : ```10```, ```20```, ```30```, ```40``` et ```50``` respectivement. Il existe un sixième niveau : ```NOTSET``` / ```0``` qui est uniquement utilisé pour donner une valeur par défaut aux niveaux des *loggers* / *handlers*, un message ne pouvant avoir de niveau inférieur à ```DEBUG``` / ```10```. 

### La hiérarchie des loggers
Chaque *logger* a un nom. Comme en Java, ceux-ci peuvent se ranger dans une hiérarchie où chaque *logger* possède un parent et possiblement un ou plusieurs enfants. Cette hiérarchie se lit dans le nom complet de chaque *logger* qui comprend le nom du logger lui-même ainsi que ceux de tous ses parents séparés par un ```.```. Le *logger* ```foo.bar``` est ainsi l'enfant du *logger* ```foo``` et le parent du *logger* ```foo.bar.baz```.

Un objet ```Logger``` se récupère via ```logging.getLogger('loggerName')```. L'implémentation de ```Logger``` est un singleton : de multiples appels de ```getLogger('loggerName')``` retournent toujours le même objet. Afin d'éviter tout problème de nommage du *logger* et de profiter de la hiérarchie des *loggers*, on recommande de récupérer son logger ainsi : ```logging.getLogger(__name__)```. Cette solution fournit un *logger* par module Python et assure que tous les noms sont de la forme ```foo.bar.baz``` car comme pour les *loggers*, les modules portent dans leur nom complet les noms du module lui-même mais aussi ceux des packages et sous-packages dont ils font partie séparés par un ```.```.

Il convient de distinguer un *logger* particulier : le *root logger* (nommé ```root```) qui est le seul à ne pas posséder de parent et dont tous les *loggers* héritent. On peut le récupère via ```logging.getLogger()```. On peut également *logger* via les *module-level functions* ```logging.info(msg)```, ```logging.debug(msg)```, etc. et qui équivalent à utiliser le *root logger*. 

### Le *logging flow*
Pour qu'un appel à une méthode d'un objet ```Logger``` débouche sur l'émission concrète d'un message par un ```Handler``` vers sa destination, une succession de conditions correspondant au *logging flow* doivent être remplies. Pour un appel à une méthode du *logger* ```logger```, si le message ne satisfait pas à chaque étape du *logging flow*, sa propagation d'interrompt et aucun message n'est finalement émis. L'appel à une méthode d'un ```logger``` se suit ainsi des contrôles suivant :  
1. Le niveau de sévérité associé à la méthode est-il supérieur à celui défini pour le *logger* courant ? Si le niveau du *logger* n'a pas pas été défini, on hérite du niveau du plus proche parent pour lequel cela a été fait. 
2. Le message satifait-il aux éventuels filtres (aucun par défaut) attachés à au *logger* courant ?
3. Le *logger* courant possède-t-il un handler ? Si oui, le message lui est passé : cf. les contrôles effectués par le handler.
4. L'attribut ```propagate``` du *logger* courant prend-il pour valeur ```True``` (valeur par défaut) et le logger courant n'est pas le *root logger* ? Si oui le message est passé au *logger* parent et on reprend la procédure en 1.

En cas de passage d'un message à un *handler*, on évalue son émission que :
1. Le niveau de sévérité du message est supérieur ou égal à celui défini pour le *handler* courant (```NOTSET``` / ```0``` par défaut).
2. Le message satisfait aux éventuels filtres attachés au *handler* (aucun par défaut). 

Une fois toutes ces conditions satisfaites le message est formatté et émis par le *handler* vers sa destination.

En réalité après un appel au *logger*, on contrôle le niveau de sévérité. Si la condition est satisfaite le "message" (un objet ```LogRecord```) est créé (les filtres agissant d'ailleurs sur cet objet). Cet objet ```LogRecord``` est ensuite propagé et examiné au fil de sa propagation jusqu'au(x) *handler(s)* chargé(s) de l'émettre vers sa ou ses destination(s).

Quelques remarques importantes : 
* Par défaut, le niveau de sévérité des *loggers* n'est définit que pour le *root logger* et fixé à ```WARNING``` / ```30```. Pour les autres loggers il est fixé à ```NOTSET``` / ```0```. Le niveau de sévérité d'un *logger* est fixé via le *setter* ```Logger.setLevel()```.
* Par défaut, le niveau de sévérité d'un *handler* ```NOTSET``` / ```0```.
* Par défaut, aucun logger ne possède de *handler* (et chacun peut en posséder un ou plusieurs) mais suivant les environnements le *logging* peut être implicitement configuré. Le *root logger* sera alors souvent le seul à posséder un unique *handler* qui émet vers la sortie standard ou d'erreur.
* Par essence, ni *loggers* ni *handlers* ne possèdent de filtres par défaut, ces derniers étant définis par l'utilisateur.
* **Propagation des messages** : Que le logger courant possède des *handlers* ou non, un message qu'il ou un de ses enfant a émis sera toujours propagé à ses parents et éventuellement émis par leurs handlers. Ce comportement est contrôlé par l'attribut ```propagate``` qui est à ```True``` par défaut. L'activation par défaut de ce comportement a l'avantage qu'on a pas à attacher de handler pour chacun de nos *loggers* : définir un unique *handler* pour le *root logger* peut ainsi être suffisant. Ce comportement par défaut est suffisant pour une large variété d'usage et économise beaucoup de *boilerplate code*. La propagation peut en revanche devenir nuisible dans les cas où on a des *handlers* à différents niveau de la hiérarchie : afin d'éviter qu'un message ne soit émis deux fois ou plus on peut désactiver la propagation au niveau de la chaine le plus approprié. 


---------------


Remarque : il existe une méthode particulière : Logger.exception() ne pouvant être utilisée que depuis un exception handler et qui correspond à logging.error avec la fonctionalité supplémentaire de remonter la stack trace de l'exception. Exemple :

```python
try:
    x = 1 / 0
except ZeroDivisionError as e:
    logging.exception('ZeroDivisionError: %s', e)
```

#### Handlers : 
Les objets ```Handlers``` reçoivent des logs du *logger* auquel ils sont associés pourvu que les messages satisfassent aux critères de sévérité (*effective level*, ```Filters```) fixés pour ce logger. Les objets ```Handlers``` proposent comme pour les *loggers* *effective level* et ```Filters``` permettant un contrôle supplémentaire sur les messages finalement envoyés à une destination partiulière. Dans le cas d'un *logger* avec deux *handlers* par exemple, on pourrait souhaiter que les messages de sévérité supérieure ou égale à ```WARNING``` soient écrits dans un fichiers mais que les messages ```ERROR``` uniquement soient envoyés par mail à un destinataire. La possibilité de filtrer également au niveau des *handlers* rend possible l'implémentation d'un tel comportement.

A chaque type de destination correspond un *handler*, les plus fréquents étant le ```StreamHandler``` qui envoie les messages vers des *streams* (par défaut ```sys.stderr```) et le ```FileHandler``` qui écrit les messages dans des fichiers texte.

Le formattage des messages finalement écrit se gère également au niveau du *handler* (```setFormatter()```), autorisant potentiellement un formattage par destination.

cf. https://docs.python.org/2/howto/logging.html#useful-handlers

#### Formatters
Les ```Formatters``` permettent de contrôler l'ordre et l'aspect des éléments constitutifs d'un message que sont notamment :
* Le *timestamp*
* Le niveau de sévérité
* Le message lui-même 

#### Filters
Qu'ils soient définis pour un *logger ou un *handler*, les filtres permettent d'implémenter une très grande variété de comportements de filtrage (on peut globalement faire ce que l'on veut) et autorisent ainsi une très grande finesse dans le filtrage des messages permettent de faire des opérations assez complexes. Cependant, la grande flexibilité qu'ils offrent vient au prix d'une implémentation plus complexe : la création d'un filtre passe par la création d'une classe héritant de ```logging.Filter``` et la réimplmentetation de la méthode ```filter()``` en respectant un certain protocole. 

Donner un exemple 

Possible d'importer des filtres prédéfinis dans un fichier / dict de conf ?

#### Loggers
Un objet logger a trois objectifs :

    Exposer un ensemble de méthodes permettent de logger l'application utilisatrice à l'exécution.
    Déterminer si un message produit doit être propagé suivant le degré de sévérité de celui-ci. Se contrôle à l'aide d'un niveau minimal de logging (fixé via Logger.setLevel()) ou plus finement à l'aide d'un objet Filter (Logger.addFilter() / Logger.removeFilter()). On désigne ce niveau minimal de sévérité par le terme d'effective level. Si le niveau n'a pas été explicitement défini, alors un logger héritera de celui de son parent et ainsi de suite. Le root logger possède toujours un effective level réglé à WARN par défaut.
    Passer le message aux objets Handlers (Logger.addHandler() / Logger.removeHandler()) pertinents si le niveau de sévérité du message est approprié. Par défaut un logger propage ses messages aux handlers de ses parents. Il n'est ainsi pas obligatoire de définir un ou des handlers pour chaque logger. Un handler définit au sommet de la hiérarchie (le root logger) suffira la plupart du temps. On peut désactiver cette propagation en fixant l'attribut propagate du logger qu'on souhaite voir bloquer la propagation des messages vers le haut sur False.


#### Configuration du logging
On peut la faire de trois façons : 
* On instancie et configure explicitement dans le code les objets Logger, Handler et Formatter.
* On crée un fichier de configuration (analogue au log4j.properties) qu'on lit à l'aide de ```logging.config.fileConfig('logging.conf')```. On a ensuite plus qu'à récupérer les loggers et le code en est allégé. On peut en effet considérer peu pratique et peu élégant d'avoir les configurations de ses loggers dispersées dans son code (*file-based configuration*). 
* Comme ci-dessus mais la configuration est contenue dans un dictionnaire qui est passé à ```logging.config.dictConfig(cfg)``` (*dictionary-based configuration*).

Si on ne se borne qu'à un logging rudimentaire (un root logger redirigé vers la console par exemple), on peut aussi simplement configurer le logging via ```logging.basicConfig()```. 

Remarque : A quel endroit du code placer l'instruction de configuration du logging ?

Remarque : Si on souhaite changer le logging level, le formattage ou ajouter un handler pour un bloc de code uniquement, on peut élégamment le faire en utilisant un context manager dont la méthode ```__enter__``` se charge de modifier le logger et ```__exit__``` de rétablir l'ancienne configuration.

```python
import logging

# Create logger
module_logger = logging.getLogger(__name__)

# Set effective level
module_logger.setLevel(logging.DEBUG)
# Create and configure a file handler for DEBUG+ logs
fh = logging.FileHandler('logs.log')
fh.setLevel(logging.DEBUG)
# Create and configure a file handler for ERROR+ logs
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# Create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# Add the handlers to the logger
module_logger.addHandler(fh)
module_logger.addHandler(ch)

module_logger.info('log something')

class MyClass:
    def __init__(self):
        self.logger = logging.getLogger(__name__ + '.MyClass')
        self.logger.info('creating an instance of MyClass')

    def do_something(self):
        self.logger.info('doing something')
    
```

Logs de Ipyhon ?
Logger dans Spark ? Est-ce utile ?

In [2]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel('INFO')
shandler = logging.StreamHandler(stream=sys.stdout)
logger.addHandler(shandler)

In [22]:
logger.info('Hello')

Hello


In [1]:
import logging
vars(logging.getLogger())

{'filters': [],
 'name': 'root',
 'level': 30,
 'parent': None,
 'propagate': True,
 'handlers': [],
 'disabled': False,
 '_cache': {}}

In [3]:
vars(logger)

{'filters': [],
 'name': 'root',
 'level': 20,
 'parent': None,
 'propagate': True,
 'handlers': [<StreamHandler stdout (NOTSET)>],
 'disabled': False,
 '_cache': {}}